# Discrete Event Simulation and Optimization Problems using Python Libraries

This notebook presents two applied analytics projects that leverage simulation and optimization techniques to address real-world operational challenges in service and supply chain domains.

## 🔹 1: Discrete Event Simulation – Call Centre Performance

A simulated call centre environment is modeled to analyze daily operations over a year. The simulation captures key service metrics such as call arrival, wait, and service times to evaluate performance against target benchmarks. This model helps identify operational inefficiencies and supports decision-making for service improvement.

## 🔹 2: Mixed Integer Linear Programming – Supply Chain Optimization

An optimization model is developed for a company producing and distributing three products to two regional warehouses. Considering constraints like production capacity, material availability, shipping costs, storage limits, and market balance, the model identifies an optimal plan that maximizes profit.

Together, these projects illustrate the power of analytics in enhancing operational performance through data-driven modeling and optimization.

# 1: Discrete Event Simulation

## Call Centre Queue Simulation

A simulated call centre dataset and notebook, designed to be used as a classroom / tutorial dataset for Business and Operations Analytics.

This notebook and dataset details the creation of simulated call centre logs over the course of one year. For this dataset we are imagining a business whose lines are open from 8:00am to 6:00pm, Monday to Friday. Four agents are on duty at any given time and each call takes an average of 5 minutes to resolve.

The call centre manager is required to meet a performance target: 90% of calls must be answered within 1 minute. Lately, the performance has slipped. As the data analytics expert, you have been brought in to analyze their performance and make recommendations to return the centre back to its target.

The dataset records timestamps for when a call was placed, when it was answered, and when the call was completed. The total waiting and service times are calculated, as well as a logical for whether the call was answered within the performance standard.

Discrete-Event Simulation allows us to model real calling behaviour with a few simple variables.

Arrival Rate
Service Rate
Number of Agents

In [1]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from datetime import datetime
import simpy
import random

df_calls = pd.read_csv('simulated_call_centre.csv')

def time_to_minutes(t):
    dt = datetime.strptime(t, '%I:%M:%S %p')
    return dt.hour * 60 + dt.minute + dt.second / 60

df_calls['arrival_min'] = df_calls['call_started'].apply(time_to_minutes)
df_calls = df_calls.sort_values('arrival_min').reset_index(drop=True)

df_calls['service_time'] = df_calls['service_length'] / 60
df_calls['wait_time'] = df_calls['wait_length'] / 60

df_calls.head()

,call_id,date,daily_caller,call_started,call_answered,call_ended,wait_length,service_length,meets_standard,arrival_min,service_time,wait_time
0,1,2021-01-01,1,8:00:00 AM,8:00:00 AM,8:14:22 AM,0,863,True,480.0,14.383333,0.0
1,17334,2021-06-03,1,8:00:00 AM,8:00:00 AM,8:03:46 AM,0,226,True,480.0,3.766667,0.0
2,30518,2021-09-06,1,8:00:00 AM,8:00:00 AM,8:02:59 AM,0,179,True,480.0,2.983333,0.0
3,17520,2021-06-04,1,8:00:00 AM,8:00:00 AM,8:06:38 AM,0,399,True,480.0,6.650000,0.0
4,17722,2021-06-07,1,8:00:00 AM,8:00:00 AM,8:00:53 AM,0,54,True,480.0,0.900000,0.0


In [3]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51708 entries, 0 to 51707
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   call_id         51708 non-null  int64  
 1   date            51708 non-null  object 
 2   daily_caller    51708 non-null  int64  
 3   call_started    51708 non-null  object 
 4   call_answered   51708 non-null  object 
 5   call_ended      51708 non-null  object 
 6   wait_length     51708 non-null  int64  
 7   service_length  51708 non-null  int64  
 8   meets_standard  51708 non-null  bool   
 9   arrival_min     51708 non-null  float64
 10  service_time    51708 non-null  float64
 11  wait_time       51708 non-null  float64
dtypes: bool(1), float64(3), int64(4), object(4)
memory usage: 4.4+ MB


In [4]:
''' Defines simulation parameters such as agents, service times, and total working minutes in a year. '''

NUM_AGENTS = 4
TARGET_WAIT_TIME = 1 
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

results = []

In [5]:
''' Simulates a single call’s behavior—waiting for an agent and receiving service—and records the metrics.'''

def call(env, call_id, agents, service_time):
    arrival_time = env.now
    with agents.request() as request:
        yield request
        wait_time = env.now - arrival_time
        yield env.timeout(service_time)
        
        results.append({
            'call_id': call_id,
            'arrival_time': arrival_time,
            'wait_time': wait_time,
            'service_time': service_time,
            'meets_standard': wait_time <= TARGET_WAIT_TIME
        })

In [6]:
''' Generates calls over time based on a Poisson process and initiates their simulation. '''

def call_generator(env, agents, call_schedule):
    previous_arrival = 0
    for i, row in call_schedule.iterrows():
        arrival = row['arrival_min']
        service_time = row['service_time']
        
        yield env.timeout(arrival - previous_arrival)
        
        env.process(call(env, i+1, agents, service_time))
        
        previous_arrival = arrival

In [7]:
''' Sets up and runs the full call center simulation, returning the results as a DataFrame. '''

def run_simulation_from_csv(call_data):
    env = simpy.Environment()
    agents = simpy.Resource(env, capacity=NUM_AGENTS)
    env.process(call_generator(env, agents, call_data))
    sim_duration = call_data['arrival_min'].max() + call_data['service_time'].max() + 10
    env.run(until=sim_duration)
    return pd.DataFrame(results)

In [8]:
# Run simulation
df_results = run_simulation_from_csv(df_calls)

In [9]:
# Performance metrics
answered_within_1_min = df_results['meets_standard'].mean() * 100
avg_wait = df_results['wait_time'].mean()
avg_service = df_results['service_time'].mean()

print(f"Total Calls Simulated: {len(df_results)}")
print(f"Percentage Answered Within 1 Minute: {answered_within_1_min:.2f}%")
print(f"Average Wait Time: {avg_wait:.2f} mins")
print(f"Average Service Time: {avg_service:.2f} mins")

Total Calls Simulated: 527
Percentage Answered Within 1 Minute: 0.76%
Average Wait Time: 323.58 mins
Average Service Time: 4.95 mins


## 📊 Results and Discussion

**Summary of Results:**

- **Total Calls Simulated:** 527  
- **Percentage Answered Within 1 Minute:** 0.76%  
- **Average Wait Time:** 323.58 minutes (~5.4 hours)  
- **Average Service Time:** 4.95 minutes  

---

## 📌 Conclusion

The call center performance remains **critically underperforming**. Key observations include:

- A **very low percentage (0.76%)** of calls answered within 1 minute, showing minimal improvement.
- An **average wait time exceeding 5 hours** continues to be unacceptable, indicating persistent and significant delays.
- The **average service time of 4.95 minutes** per call remains reasonable but does little to offset the long wait times.

---

## ⚠️ Implications

- High risk of **customer dissatisfaction** due to prolonged waiting periods.
- Possible underlying issues:
  - **Understaffing**
  - **Ineffective call scheduling**
  - **Suboptimal queuing and call routing**
- Immediate focus areas include:
  - Adjusting staffing and workforce allocation
  - Optimizing queue management strategies
  - Considering automation or self-service options
  - Implementing better workload distribution mechanisms

---

## ✅ Recommendation

A comprehensive operational review is urgently needed to:

- Improve responsiveness and reduce wait times
- Enhance overall customer satisfaction and experience
- Restore reliability and quality of call center services

# 2: Linear Programming for Optimization

## Question: Multi-Product Manufacturing and Distribution Optimization

### Problem Description

A company manufactures **3 products**: Product A, Product B, and Product C. These products are produced in a single factory with limited resources and then shipped to **2 different regional warehouses** before reaching customers.

---

### Data

- **Profit per unit:**
  - Product A: \$40
  - Product B: \$50
  - Product C: \$55

- **Production time required per unit:**
  - Product A: 2 hours
  - Product B: 3 hours
  - Product C: 4 hours

- **Material required per unit (kg):**
  - Product A: 5 kg
  - Product B: 7 kg
  - Product C: 6 kg

- **Available resources per week:**
  - Total production time: 2400 hours
  - Raw material: 4000 kg

- **Demand constraints (units per week):**
  - Warehouse 1:
    - Product A: at most 300 units
    - Product B: at most 200 units
    - Product C: at most 250 units
  - Warehouse 2:
    - Product A: at most 200 units
    - Product B: at most 300 units
    - Product C: at most 300 units

- **Shipping cost per unit from factory to warehouses:**
  - Product A:
    - To Warehouse 1: \$5
    - To Warehouse 2: \$7
  - Product B:
    - To Warehouse 1: \$6
    - To Warehouse 2: \$5
  - Product C:
    - To Warehouse 1: \$4
    - To Warehouse 2: \$6

- **Warehouse storage cost per unit per week:**
  - Warehouse 1: \$2
  - Warehouse 2: \$3

- **Maximum storage capacity of warehouses:**
  - Warehouse 1: 600 units total (sum of all products)
  - Warehouse 2: 700 units total (sum of all products)

- **Additional constraint:**  
  - At least **40%** of total units shipped must be to Warehouse 1 (to maintain market balance).

### Decision Variables

Let:

\[
\begin{align*}
x_{A1} &= \text{units of Product A shipped to Warehouse 1} \\
x_{A2} &= \text{units of Product A shipped to Warehouse 2} \\
x_{B1} &= \text{units of Product B shipped to Warehouse 1} \\
x_{B2} &= \text{units of Product B shipped to Warehouse 2} \\
x_{C1} &= \text{units of Product C shipped to Warehouse 1} \\
x_{C2} &= \text{units of Product C shipped to Warehouse 2} \\
\end{align*}
\]

---

### Objective Function

Maximize total profit:

\[
\max Z = \sum_{i \in \{A,B,C\}} \sum_{j \in \{1,2\}} \Big[ (\text{Profit}_i) \cdot x_{ij} - (\text{ShippingCost}_{ij}) \cdot x_{ij} - (\text{StorageCost}_j) \cdot x_{ij} \Big]
\]

Explicitly,

\[
\begin{aligned}
\max Z = \quad & (40 - 5 - 2) x_{A1} + (40 - 7 - 3) x_{A2} + \\
& (50 - 6 - 2) x_{B1} + (50 - 5 - 3) x_{B2} + \\
& (55 - 4 - 2) x_{C1} + (55 - 6 - 3) x_{C2}
\end{aligned}
\]

---

### Constraints

1. **Production time:**

\[
2(x_{A1} + x_{A2}) + 3(x_{B1} + x_{B2}) + 4(x_{C1} + x_{C2}) \leq 2400
\]

2. **Material availability:**

\[
5(x_{A1} + x_{A2}) + 7(x_{B1} + x_{B2}) + 6(x_{C1} + x_{C2}) \leq 4000
\]

3. **Demand limits per warehouse:**

\[
\begin{cases}
x_{A1} \leq 300, & x_{A2} \leq 200 \\
x_{B1} \leq 200, & x_{B2} \leq 300 \\
x_{C1} \leq 250, & x_{C2} \leq 300 \\
\end{cases}
\]

4. **Warehouse capacity:**

\[
x_{A1} + x_{B1} + x_{C1} \leq 600
\]
\[
x_{A2} + x_{B2} + x_{C2} \leq 700
\]

5. **Market balance:**

\[
x_{A1} + x_{B1} + x_{C1} \geq 0.4 \times \big( x_{A1} + x_{A2} + x_{B1} + x_{B2} + x_{C1} + x_{C2} \big)
\]

6. **Non-negativity:**

\[
x_{ij} \geq 0, \quad \forall i \in \{A,B,C\}, \; j \in \{1,2\}
\]

---

### Tasks

1. Formulate the full linear programming model (objective + constraints).  
2. Solve for the optimal shipment quantities \(x_{ij}\) to maximize profit.  
3. Analyze how the solution changes if the available production time reduces to 1800 hours.  
4. Discuss the impact if Warehouse 1 storage capacity reduces to 400 units.  

## Solution: Using Gurobi

In [10]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [11]:
from gurobipy import Model, GRB, quicksum

def solve_manufacturing_distribution():
    # Create a new model
    model = Model("Manufacturing_Distribution_Optimization")

    # Products and Warehouses
    products = ['A', 'B', 'C']
    warehouses = ['1', '2']

    # Parameters
    
    # Profit per unit for products
    profit = {'A': 40, 'B': 50, 'C': 55}

    # Production time per unit (hours)
    prod_time = {'A': 2, 'B': 3, 'C': 4}

    # Material required per unit (kg)
    material = {'A': 5, 'B': 7, 'C': 6}

    # Resource limits
    total_prod_time = 2400
    total_material = 4000

    # Demand limits (max units per warehouse)
    demand_limits = {
        ('A', '1'): 300, ('A', '2'): 200,
        ('B', '1'): 200, ('B', '2'): 300,
        ('C', '1'): 250, ('C', '2'): 300,
    }

    # Shipping cost per unit from factory to warehouses
    shipping_cost = {
        ('A', '1'): 5, ('A', '2'): 7,
        ('B', '1'): 6, ('B', '2'): 5,
        ('C', '1'): 4, ('C', '2'): 6,
    }

    # Warehouse storage cost per unit per week
    storage_cost = {'1': 2, '2': 3}

    # Warehouse storage capacity (total units)
    storage_capacity = {'1': 600, '2': 700}

    # Decision variables: units shipped of product i to warehouse j
    x = {}
    for i in products:
        for j in warehouses:
            x[i,j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"x_{i}{j}", lb=0)

    model.update()

    # Objective: Maximize total profit
    # Profit per unit - shipping cost per unit - storage cost per unit
    objective = quicksum(
        (profit[i] - shipping_cost[i,j] - storage_cost[j]) * x[i,j]
        for i in products for j in warehouses
    )
    model.setObjective(objective, GRB.MAXIMIZE)

    # Constraints

    # 1. Production time constraint
    model.addConstr(
        quicksum(prod_time[i] * (x[i,'1'] + x[i,'2']) for i in products) <= total_prod_time,
        name="ProductionTime"
    )

    # 2. Material availability constraint
    model.addConstr(
        quicksum(material[i] * (x[i,'1'] + x[i,'2']) for i in products) <= total_material,
        name="Material"
    )

    # 3. Demand limits for each product and warehouse
    for i in products:
        for j in warehouses:
            model.addConstr(x[i,j] <= demand_limits[(i,j)], name=f"DemandLimit_{i}{j}")

    # 4. Warehouse storage capacity constraints
    for j in warehouses:
        model.addConstr(
            quicksum(x[i,j] for i in products) <= storage_capacity[j],
            name=f"StorageCapacity_{j}"
        )

    # 5. Market balance constraint
    # Total shipped to warehouse 1 >= 40% of total shipped units
    total_to_wh1 = quicksum(x[i,'1'] for i in products)
    total_shipped = quicksum(x[i,j] for i in products for j in warehouses)
    model.addConstr(
        total_to_wh1 >= 0.4 * total_shipped,
        name="MarketBalance"
    )

    # Optimize the model
    model.optimize()

    # Output results
    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found:\n")
        print("Shipment Plan (units):")
        for i in products:
            for j in warehouses:
                val = x[i,j].X
                print(f"Product {i} to Warehouse {j}: {val:.2f}")

        print(f"\nMaximum Profit: ${model.objVal:.2f}")

        # Sensitivity analysis (Optional)
        print("\nShadow Prices / Dual values for constraints:")
        for c in model.getConstrs():
            print(f"{c.ConstrName}: dual = {c.Pi:.4f}, slack = {c.Slack:.2f}")
    else:
        print("No optimal solution found.")

if __name__ == "__main__":
    solve_manufacturing_distribution()

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 6 columns and 30 nonzeros
Model fingerprint: 0x8e8bdca4
Coefficient statistics:
  Matrix range     [4e-01, 7e+00]
  Objective range  [3e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 4e+03]
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9600000e+04   1.325000e+03   0.000000e+00      0s
       5    3.0350000e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.035000000e+04

Optimal solution found:

Shipment Plan (units):
Product A to Warehouse 1: 200.00
Product A to Warehouse 2: 0.00


---

**Conclusion:**

The Gurobi optimizer successfully solved the manufacturing and distribution linear programming problem in 5 iterations within 0.01 seconds, achieving an **optimal profit of \$30,350.00**. The optimal shipment plan suggests producing and shipping 200 units of Product A to Warehouse 1, no shipments of Product B, and 250 units of Product C to each Warehouse 1 and Warehouse 2. The constraints for production time (2400 hours) and material availability (4000 kg) are both fully utilized with zero slack, indicated by positive dual values of 4.0 and 5.0 respectively. Some demand limits and warehouse storage constraints remain underutilized, with slack values greater than zero, meaning there is still unused capacity for those parameters. The market balance constraint is not binding, as reflected by a zero dual value and a negative slack. Overall, the model efficiently balances production, material, demand, storage, and market constraints to maximize profit under the defined conditions.

## Solution: Using Google OR Tools

In [1]:
pip install --upgrade ortools

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ortools.linear_solver import pywraplp

def solve_manufacturing_distribution_or_tools():
    # Create the solver
    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        print("CBC solver unavailable.")
        return

    # Products and Warehouses
    products = ['A', 'B', 'C']
    warehouses = ['1', '2']

    # Parameters
    profit = {'A': 40, 'B': 50, 'C': 55}
    prod_time = {'A': 2, 'B': 3, 'C': 4}
    material = {'A': 5, 'B': 7, 'C': 6}
    total_prod_time = 2400
    total_material = 4000
    demand_limits = {
        ('A', '1'): 300, ('A', '2'): 200,
        ('B', '1'): 200, ('B', '2'): 300,
        ('C', '1'): 250, ('C', '2'): 300,
    }
    shipping_cost = {
        ('A', '1'): 5, ('A', '2'): 7,
        ('B', '1'): 6, ('B', '2'): 5,
        ('C', '1'): 4, ('C', '2'): 6,
    }
    storage_cost = {'1': 2, '2': 3}
    storage_capacity = {'1': 600, '2': 700}

    # Decision variables: x[i,j] = units of product i shipped to warehouse j
    x = {}
    for i in products:
        for j in warehouses:
            x[i,j] = solver.NumVar(0, solver.infinity(), f'x_{i}{j}')

    # Objective: maximize total profit = sum over (profit - shipping cost - storage cost) * x[i,j]
    objective = solver.Objective()
    for i in products:
        for j in warehouses:
            unit_profit = profit[i] - shipping_cost[i,j] - storage_cost[j]
            objective.SetCoefficient(x[i,j], unit_profit)
    objective.SetMaximization()

    # Constraints

    # 1. Production time constraint
    production_time_expr = solver.Sum(prod_time[i] * (x[i,'1'] + x[i,'2']) for i in products)
    solver.Add(production_time_expr <= total_prod_time)

    # 2. Material availability constraint
    material_expr = solver.Sum(material[i] * (x[i,'1'] + x[i,'2']) for i in products)
    solver.Add(material_expr <= total_material)

    # 3. Demand limits constraints
    for i in products:
        for j in warehouses:
            solver.Add(x[i,j] <= demand_limits[(i,j)])

    # 4. Warehouse storage capacity constraints
    for j in warehouses:
        solver.Add(solver.Sum(x[i,j] for i in products) <= storage_capacity[j])

    # 5. Market balance constraint
    total_to_wh1 = solver.Sum(x[i,'1'] for i in products)
    total_shipped = solver.Sum(x[i,j] for i in products for j in warehouses)
    solver.Add(total_to_wh1 >= 0.4 * total_shipped)

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("\nOptimal solution found:\n")
        print("Shipment Plan (units):")
        for i in products:
            for j in warehouses:
                print(f"Product {i} to Warehouse {j}: {x[i,j].solution_value():.2f}")

        print(f"\nMaximum Profit: ${objective.Value():.2f}")

        # Google OR-Tools does not provide duals or shadow prices in CBC solver.
        print("\nNote: Dual values / shadow prices are not available with OR-Tools CBC solver.")
    else:
        print("No optimal solution found.")

if __name__ == "__main__":
    solve_manufacturing_distribution_or_tools()


Optimal solution found:

Shipment Plan (units):
Product A to Warehouse 1: 200.00
Product A to Warehouse 2: 0.00
Product B to Warehouse 1: 0.00
Product B to Warehouse 2: 0.00
Product C to Warehouse 1: 250.00
Product C to Warehouse 2: 250.00

Maximum Profit: $30350.00

Note: Dual values / shadow prices are not available with OR-Tools CBC solver.


---

**Conclusion:**

The optimization results indicate that the most profitable manufacturing and distribution strategy involves producing and shipping 200 units of Product A to Warehouse 1, and 250 units of Product C each to Warehouses 1 and 2, while entirely excluding Product B from production. This strategy yields a maximum profit of $30,350, achieved by efficiently utilizing available production time and material resources, and by prioritizing products with the highest net returns after accounting for shipping and storage costs. The solution also satisfies all operational constraints, including warehouse storage limits and the market balance requirement that mandates at least 40% of total shipments be directed to Warehouse 1. The absence of Product B in the shipment plan suggests it is less cost-effective under the given constraints, reinforcing the importance of cost-benefit analysis in resource-limited production planning.

## Solution: Using PuLP

In [5]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 7.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus

def solve_manufacturing_distribution_pulp():
    # Create a problem instance
    model = LpProblem("Manufacturing_Distribution_Optimization", LpMaximize)

    # Products and Warehouses
    products = ['A', 'B', 'C']
    warehouses = ['1', '2']

    # Parameters
    profit = {'A': 40, 'B': 50, 'C': 55}
    prod_time = {'A': 2, 'B': 3, 'C': 4}
    material = {'A': 5, 'B': 7, 'C': 6}
    total_prod_time = 2400
    total_material = 4000

    demand_limits = {
        ('A', '1'): 300, ('A', '2'): 200,
        ('B', '1'): 200, ('B', '2'): 300,
        ('C', '1'): 250, ('C', '2'): 300,
    }

    shipping_cost = {
        ('A', '1'): 5, ('A', '2'): 7,
        ('B', '1'): 6, ('B', '2'): 5,
        ('C', '1'): 4, ('C', '2'): 6,
    }

    storage_cost = {'1': 2, '2': 3}
    storage_capacity = {'1': 600, '2': 700}

    # Decision variables
    x = {
        (i, j): LpVariable(f"x_{i}_{j}", lowBound=0, cat='Continuous')
        for i in products for j in warehouses
    }

    # Objective function
    model += lpSum(
        (profit[i] - shipping_cost[i, j] - storage_cost[j]) * x[i, j]
        for i in products for j in warehouses
    ), "Total_Profit"

    # Constraints

    # 1. Production time constraint
    model += lpSum(prod_time[i] * (x[i, '1'] + x[i, '2']) for i in products) <= total_prod_time, "ProductionTime"

    # 2. Material availability constraint
    model += lpSum(material[i] * (x[i, '1'] + x[i, '2']) for i in products) <= total_material, "Material"

    # 3. Demand limits
    for i in products:
        for j in warehouses:
            model += x[i, j] <= demand_limits[i, j], f"DemandLimit_{i}_{j}"

    # 4. Storage capacity
    for j in warehouses:
        model += lpSum(x[i, j] for i in products) <= storage_capacity[j], f"StorageCapacity_{j}"

    # 5. Market balance: WH1 receives at least 40% of total shipment
    total_to_wh1 = lpSum(x[i, '1'] for i in products)
    total_shipped = lpSum(x[i, j] for i in products for j in warehouses)
    model += total_to_wh1 >= 0.4 * total_shipped, "MarketBalance"

    # Solve the model
    model.solve()

    # Output results
    print(f"\nSolver Status: {LpStatus[model.status]}")
    if LpStatus[model.status] == 'Optimal':
        print("\nOptimal solution found:\n")
        print("Shipment Plan (units):")
        for i in products:
            for j in warehouses:
                print(f"Product {i} to Warehouse {j}: {x[i,j].varValue:.2f}")

        print(f"\nMaximum Profit: ${model.objective.value():.2f}")
    else:
        print("No optimal solution found.")

if __name__ == "__main__":
    solve_manufacturing_distribution_pulp()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/vn/8zws5yln68766mf_fnhry1940000gn/T/4ec40a6e39f74d5388ab899276a85058-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/vn/8zws5yln68766mf_fnhry1940000gn/T/4ec40a6e39f74d5388ab899276a85058-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 53 RHS
At line 65 BOUNDS
At line 66 ENDATA
Problem MODEL has 11 rows, 6 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (-6) rows, 6 (0) columns and 24 (-6) elements
0  Obj -0 Dual inf 241.99999 (6)
2  Obj 30350
Optimal - objective value 30350
After Postsolve, objective 30350, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 30350 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed

---

**Conclusion:**

The optimization model successfully identified the most profitable manufacturing and distribution plan using PuLP, with a maximum achievable profit of $30,350. The solution suggests producing and shipping 200 units of Product A to Warehouse 1, 250 units of Product C to Warehouse 1, and 250 units of Product C to Warehouse 2, while Products B and A to Warehouse 2 are not shipped at all. This distribution meets all the resource constraints—production time and material limits—along with demand limits and warehouse capacities. Moreover, it satisfies the market balance condition by ensuring that at least 40% of the total shipped units are sent to Warehouse 1. This result highlights the efficiency of optimization techniques in maximizing profit while respecting operational limitations in a supply chain scenario.